In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm_notebook as tn 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype

        if col_type not in ['object', 'datetime64[ns]']:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df





In [ ]:
train= pd.read_csv('/kaggle/input/train.csv')
payment_history = pd.read_csv('/kaggle/input/payment_history.csv')
#client_data = pd.read_csv('/kaggle/input/client_data.csv')
sub = pd.read_csv('/kaggle/input/sample_sub.csv')
test = pd.read_csv('/kaggle/input/sample_sub.csv')
policy_data = pd.read_csv('/kaggle/input/policy_data.csv')
train['Lapse' ] = np.where( ( train.Lapse == "?" ) & ( train['Lapse Year'] == "?" ), 0,1)
train.drop(['Lapse Year'], axis=1, inplace=True)
test.drop(['Lapse'], axis=1,inplace=True)




In [ ]:
train=train.astype(object)
test=test.astype(object)
train=reduce_mem_usage(train)
test=reduce_mem_usage(test)

In [ ]:
def Lastname(x):
    x = str(x)
    a = x.split('_')
    return a[2]
def adresse(x):
    x = str(x)
    a = x.split('_')
    return a[1]
def date_from(x):
    x=str(x)
    a = x.split(' ')
    return a[0]
def time_from(x):
    x=str(x)
    a = x.split(' ')
    return a[1]

In [ ]:
upper_lim = policy_data['NPR_PREMIUM'].quantile(.95)
lower_lim = policy_data['NPR_PREMIUM'].quantile(.05)
policy_data.loc[(policy_data['NPR_PREMIUM'] > upper_lim),'NPR_PREMIUM'] = upper_lim
policy_data.loc[(policy_data['NPR_PREMIUM'] < lower_lim),'NPR_PREMIUM'] = lower_lim

In [ ]:
upper_lim = policy_data['NPR_SUMASSURED'].quantile(.95)
lower_lim = policy_data['NPR_SUMASSURED'].quantile(.05)
policy_data.loc[(policy_data['NPR_SUMASSURED'] > upper_lim),'NPR_SUMASSURED'] = upper_lim
policy_data.loc[(policy_data['NPR_SUMASSURED'] < lower_lim),'NPR_SUMASSURED'] = lower_lim

In [ ]:
upper_lim = policy_data['NLO_AMOUNT'].quantile(.95)
lower_lim = policy_data['NLO_AMOUNT'].quantile(.05)
policy_data.loc[(policy_data['NLO_AMOUNT'] > upper_lim),'NLO_AMOUNT'] = upper_lim
policy_data.loc[(policy_data['NLO_AMOUNT'] < lower_lim),'NLO_AMOUNT'] = lower_lim

In [ ]:
upper_lim = payment_history['AMOUNTPAID'].quantile(.95)
lower_lim = payment_history['AMOUNTPAID'].quantile(.05)
payment_history.loc[(payment_history['AMOUNTPAID'] > upper_lim),'AMOUNTPAID'] = upper_lim
payment_history.loc[(payment_history['AMOUNTPAID'] < lower_lim),'AMOUNTPAID'] = lower_lim

In [ ]:
policy_data = policy_data.fillna(policy_data.median())
payment_history = payment_history.fillna(policy_data.median())

In [ ]:
policy_data.rename(columns={'NP2_EFFECTDATE':'EFFECTDATE','PPR_PRODCD':'PRODCD','NPR_PREMIUM':'PREMIUM','NPH_LASTNAME':'LASTNAME','CLF_LIFECD':'LIFECD','NSP_SUBPROPOSAL':'SUBPROPOSAL','NPR_SUMASSURED':'SUMASSURED','NLO_TYPE':'TYPE','NLO_AMOUNT':'AMOUNT'}, inplace=True)
policy_data['AGCODE'] = policy_data['AAG_AGCODE'].apply(Lastname)
policy_data['LOCATCODE'] = policy_data['PCL_LOCATCODE'].apply(Lastname)
policy_data['OCCUPATION_'] = policy_data['OCCUPATION'].apply(adresse)
policy_data['CATEGORY_'] = policy_data['CATEGORY'].apply(adresse)
policy_data['TYPE_'] = policy_data['TYPE'].apply(adresse)
policy_data.drop(['AAG_AGCODE'],axis=1, inplace=True )
policy_data.drop(['PCL_LOCATCODE'],axis=1, inplace=True )
policy_data.drop(['OCCUPATION'],axis=1, inplace=True )
policy_data.drop(['CATEGORY'],axis=1, inplace=True )
policy_data.drop(['TYPE'],axis=1, inplace=True )
policy_data['EFFECTDATE'] = pd.to_datetime(policy_data['EFFECTDATE'])
policy_data['EFFECTDATE_year'] = policy_data['EFFECTDATE'].dt.year
policy_data['EFFECTDATE_month'] = policy_data['EFFECTDATE'].dt.month
policy_data['EFFECTDATE_day'] = policy_data['EFFECTDATE'].dt.day
policy_data['EFFECTDATE_weekday'] = policy_data['EFFECTDATE'].dt.weekday

In [ ]:
EFFECTDATE_month = pd.get_dummies(policy_data['EFFECTDATE_month'] , prefix='EFFECTDATE_month')
policy_data = pd.concat([policy_data,EFFECTDATE_month],axis=1)
policy_data.drop(['EFFECTDATE_month'],axis=1 , inplace = True)

In [ ]:
del EFFECTDATE_month
gc.collect()

In [ ]:
policy_data['EFFECTDATE_dayofweek_name']=policy_data['EFFECTDATE'].dt.day_name()

In [ ]:
#OCCUPATION = pd.get_dummies(policy_data['OCCUPATION_'] , prefix='OCCUPATION')
#policy_data = pd.concat([policy_data,OCCUPATION],axis=1)
#policy_data.drop(['OCCUPATION_'],axis=1 , inplace = True)
Category = pd.get_dummies(policy_data['CATEGORY_'] , prefix='Category')
policy_data = pd.concat([policy_data,Category],axis=1)
policy_data.drop(['CATEGORY_'],axis=1 , inplace = True)
Branch_code = pd.get_dummies(policy_data['LOCATCODE'] , prefix='Branch_code')
policy_data = pd.concat([policy_data,Branch_code],axis=1)
policy_data.drop(['LOCATCODE'],axis=1 , inplace = True)


In [ ]:
policy_data['total']=policy_data['PREMIUM']+policy_data['AMOUNT']

In [ ]:
del Category , Branch_code
gc.collect()

In [ ]:
PRODCD = pd.get_dummies(policy_data['PRODCD'] , prefix='PRODCD')
policy_data = pd.concat([policy_data,PRODCD],axis=1)


In [ ]:
del PRODCD
gc.collect()

In [ ]:
NLO_TYPE = pd.get_dummies(policy_data['TYPE_'] , prefix='TYPE')
policy_data = pd.concat([policy_data,NLO_TYPE],axis=1)
policy_data.drop(['TYPE_'],axis=1 , inplace = True)
CLF_LIFECD = pd.get_dummies(policy_data['LIFECD'] , prefix='LIFECD')
policy_data = pd.concat([policy_data,CLF_LIFECD],axis=1)
policy_data.drop(['LIFECD'],axis=1 , inplace = True)
NP2_EFFECTDATE_year = pd.get_dummies(policy_data['EFFECTDATE_year'] , prefix='EFFECTDATE_year')
policy_data = pd.concat([policy_data,NP2_EFFECTDATE_year],axis=1)
policy_data.drop(['EFFECTDATE_year'],axis=1 , inplace = True)
SUBPROPOSAL = pd.get_dummies(policy_data['SUBPROPOSAL'] , prefix='SUBPROPOSAL')
policy_data = pd.concat([policy_data,SUBPROPOSAL],axis=1)
policy_data.drop(['SUBPROPOSAL'],axis=1 , inplace = True)


In [ ]:
del NLO_TYPE , CLF_LIFECD , NP2_EFFECTDATE_year 
gc.collect()

In [ ]:
policy_data['LASTNAME_']=policy_data['LASTNAME'].apply(Lastname)
policy_data['PRODCD_']=policy_data['PRODCD'].apply(Lastname)
policy_data.drop(['LASTNAME'],axis=1,inplace=True)
policy_data.drop(['PRODCD'],axis=1,inplace=True)
policy_data.rename(columns={'LASTNAME_':'LASTNAME'},inplace=True)

In [ ]:
"""client_data.fillna(method='ffill', inplace=True)
client_data.dropna(inplace=True)
client_data.rename(columns={'NPH_TITLE':'TITLE','NPH_SEX':'SEX','NPH_BIRTHDATE':'BIRTHDATE'}, inplace=True)
client_data['LASTNAME'] = client_data['NPH_LASTNAME'].apply(Lastname)
client_data['ADDRESS1'] = client_data['NAD_ADDRESS1'].apply(adresse)
client_data['ADDRESS2'] = client_data['NAD_ADDRESS2'].apply(adresse)
client_data.drop(['NPH_LASTNAME'],axis=1, inplace=True )
client_data.drop(['NAD_ADDRESS1'],axis=1, inplace=True )
client_data.drop(['NAD_ADDRESS2'],axis=1, inplace=True )
title = pd.get_dummies(client_data['TITLE'] , prefix='Title')
client_data = pd.concat([client_data,title],axis=1)
client_data.drop(['TITLE'],axis=1 , inplace = True)
Sex = pd.get_dummies(client_data['SEX'] , prefix='Sex')
client_data = pd.concat([client_data,Sex],axis=1)
client_data.drop(['SEX'],axis=1 , inplace = True)"""

In [ ]:
payment_history['PREMIUMDUEDATE'].fillna(payment_history['PREMIUMDUEDATE'].value_counts().idxmax(), inplace=True)

In [ ]:
from datetime import datetime
payment_history['DATEPAID_date'] = payment_history['DATEPAID'].apply(date_from)
payment_history['DATEPAID_time'] = payment_history['DATEPAID'].apply(time_from)
payment_history['POSTDATE_date'] = payment_history['POSTDATE'].apply(date_from)
payment_history['POSTDATE_time'] = payment_history['POSTDATE'].apply(time_from)
payment_history['PREMIUMDUEDATE_date'] = payment_history['PREMIUMDUEDATE'].apply(date_from)
payment_history['PREMIUMDUEDATE_time'] = payment_history['PREMIUMDUEDATE'].apply(time_from)
payment_history['DATEPAID'] = pd.to_datetime(payment_history['DATEPAID'])
payment_history['POSTDATE'] = pd.to_datetime(payment_history['POSTDATE'])
payment_history['PREMIUMDUEDATE'] = pd.to_datetime(payment_history['PREMIUMDUEDATE'])
payment_history['paid_year'] = payment_history['DATEPAID'].dt.year
payment_history['paid_month'] = payment_history['DATEPAID'].dt.month
payment_history['paid_day'] = payment_history['DATEPAID'].dt.day
payment_history['paid_weekday'] = payment_history['DATEPAID'].dt.weekday
payment_history['post_year'] = payment_history['POSTDATE'].dt.year
payment_history['post_month'] = payment_history['POSTDATE'].dt.month
payment_history['post_day'] = payment_history['POSTDATE'].dt.day
payment_history['post_weekday'] = payment_history['POSTDATE'].dt.weekday
payment_history['premium_year'] = payment_history['PREMIUMDUEDATE'].dt.year
payment_history['premium_month'] = payment_history['PREMIUMDUEDATE'].dt.month
payment_history['premium_day'] = payment_history['PREMIUMDUEDATE'].dt.day
payment_history['premium_weekday'] = payment_history['PREMIUMDUEDATE'].dt.weekday


In [ ]:
premium_year = pd.get_dummies(payment_history['premium_year'] , prefix='premium_year')
payment_history = pd.concat([payment_history,premium_year],axis=1)
payment_history.drop(['premium_year'],axis=1 , inplace = True)
paid_year = pd.get_dummies(payment_history['paid_year'] , prefix='paid_year')
payment_history = pd.concat([payment_history,paid_year],axis=1)
payment_history.drop(['paid_year'],axis=1 , inplace = True)
post_year = pd.get_dummies(payment_history['post_year'] , prefix='post_year')
payment_history = pd.concat([payment_history,post_year],axis=1)
payment_history.drop(['post_year'],axis=1 , inplace = True)

In [ ]:
del premium_year , paid_year , post_year
gc.collect()

In [ ]:
premium_month = pd.get_dummies(payment_history['premium_month'] , prefix='premium_month')
payment_history = pd.concat([payment_history,premium_month],axis=1)
payment_history.drop(['premium_month'],axis=1 , inplace = True)
post_month = pd.get_dummies(payment_history['post_month'] , prefix='post_month')
payment_history = pd.concat([payment_history,post_month],axis=1)
payment_history.drop(['post_month'],axis=1 , inplace = True)
paid_month = pd.get_dummies(payment_history['paid_month'] , prefix='paid_month')
payment_history = pd.concat([payment_history,paid_month],axis=1)
payment_history.drop(['paid_month'],axis=1 , inplace = True)


In [ ]:
del premium_month , post_month , paid_month 
gc.collect()

In [ ]:
payment_history['PREMIUMDUEDATE_dayofweek_name']=payment_history['PREMIUMDUEDATE'].dt.day_name()
payment_history['DATEPAID_dayofweek_name']=payment_history['DATEPAID'].dt.day_name()
payment_history['POSTDATE_dayofweek_name']=payment_history['POSTDATE'].dt.day_name()

In [ ]:
## sample train dataset
a = policy_data[['LASTNAME','OCCUPATION_']]

## Frequency Encoding title variable
b = a.groupby(['LASTNAME']).size().reset_index()
b.columns = ['LASTNAME', 'Freq_Encoded_OCCUPATION_']
policy_data = pd.merge(policy_data,b,on = 'LASTNAME',how = 'left')


In [ ]:
del a , b 
gc.collect()

In [ ]:
sample_train = policy_data[['Policy ID','AMOUNT']]
## Mean encoding 
x = sample_train.groupby(['Policy ID'])['AMOUNT'].mean().reset_index()
x = x.rename(columns={"AMOUNT" : "AMOUNT" +"_Mean_Encoded"})
policy_data = pd.merge(policy_data,x,on = 'Policy ID',how = 'left')
policy_data.head()

In [ ]:
del sample_train , x 
gc.collect()

In [ ]:
"""## sample train dataset
a = policy_data[['AGCODE','Policy ID']]

## Frequency Encoding title variable
b = a.groupby(['Policy ID']).size().reset_index()
b.columns = ['Policy ID', 'Freq_Encoded_AGCODE']
policy_data = pd.merge(policy_data,b,on = 'Policy ID',how = 'left')"""

In [ ]:
'''## sample train dataset
a = policy_data[['PRODCD_','LASTNAME']]

## Frequency Encoding title variable
b = a.groupby(['PRODCD_']).size().reset_index()
b.columns = ['PRODCD_', 'Freq_Encoded_PRODCD']
policy_data = pd.merge(policy_data,b,on = 'PRODCD_',how = 'left')'''

In [ ]:
"""## sample train dataset
a = policy_data[['SUBPROPOSAL','Policy ID']]

## Frequency Encoding title variable
b = a.groupby(['Policy ID']).size().reset_index()
b.columns = ['Policy ID', 'Freq_Encoded_SUBPROPOSAL']
policy_data = pd.merge(policy_data,b,on = 'Policy ID',how = 'left')
policy_data.head()"""

In [ ]:
'''sample_train = policy_data[['Policy ID','PREMIUM']]
## Mean encoding 
x = sample_train.groupby(['Policy ID'])['PREMIUM'].mean().reset_index()
x = x.rename(columns={"PREMIUM" : "PREMIUM" +"_Mean_Encoded"})
policy_data = pd.merge(policy_data,x,on = 'Policy ID',how = 'left')
policy_data.head()'''


In [ ]:
"""sample_train = policy_data[['Policy ID','SUMASSURED']]
## Mean encoding 
x = sample_train.groupby(['Policy ID'])['SUMASSURED'].mean().reset_index()
x = x.rename(columns={"SUMASSURED" : "SUMASSURED" +"_Mean_Encoded"})
policy_data = pd.merge(policy_data,x,on = 'Policy ID',how = 'left')
policy_data.head()"""


In [ ]:
"""sample_train = payment_history[['Policy ID','AMOUNTPAID']]
## Mean encoding 
x = sample_train.groupby(['Policy ID'])['AMOUNTPAID'].mean().reset_index()
x = x.rename(columns={"AMOUNTPAID" : "AMOUNTPAID"+"_Mean_Encoded"})
payment_history = pd.merge(payment_history,x,on = 'Policy ID',how = 'left')"""



In [ ]:
def set_cat_job(x):
    if x>0.07:
        return 0 
    elif x<0.7 and x>0.01 : 
        return 1 
    elif x<0.01 and x>0.001 : 
        return 2 
    elif x<0.001 and x>0.0001 : 
        return 3
    elif x<0.0001 and x>0.00001 : 
        return 4
    else : 
        return 5

In [ ]:
policy_data['OCCUPATION_cat']=np.zeros(policy_data.shape[0])

In [ ]:
a = list(policy_data['OCCUPATION_'].unique())
b={}
for elem in a:
    b[elem]=policy_data['OCCUPATION_'].value_counts(elem)
for i in tn(range(240)):
    a=list(b.values())[i][i]
    policy_data['OCCUPATION_cat'][i]=set_cat_job(a)

In [ ]:
occupation = pd.get_dummies(policy_data['OCCUPATION_cat'] , prefix='occupation')
policy_data = pd.concat([policy_data,occupation],axis=1)
policy_data.drop(['OCCUPATION_cat'],axis=1 , inplace = True)

In [ ]:
#policy_data.drop(['LASTNAME'],axis=1,inplace=True)
policy_data.drop(['PRODCD_'],axis=1 , inplace = True)

In [ ]:
policy_data.drop(['AGCODE','OCCUPATION_'],axis=1,inplace=True)


In [ ]:
train=train.astype(object)
test=test.astype(object)
train=reduce_mem_usage(train)
test=reduce_mem_usage(test)

In [ ]:
train = train.merge(right=policy_data.reset_index(), how='left', on='Policy ID')
train = train.merge(right=payment_history.reset_index(), how='left', on='Policy ID')
"""c.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in c.columns]
d.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in d.columns]
c.reset_index(inplace=True)
d.reset_index(inplace=True)
train = train.merge(right=c.reset_index(), how='left', on='Policy ID')
train = train.merge(right=d.reset_index(), how='left', on='Policy ID')"""

In [ ]:
test = test.merge(right=policy_data.reset_index(), how='left', on='Policy ID')
test = test.merge(right=payment_history.reset_index(), how='left', on='Policy ID')
"""c.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in c.columns]
d.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in d.columns]
c.reset_index(inplace=True)
d.reset_index(inplace=True)
c.rename(columns={'Policy_ID':'Policy ID'},inplace=True)
d.rename(columns={'Policy_ID':'Policy ID'},inplace=True)
test = test.merge(right=c.reset_index(), how='left', on='Policy ID')
test = test.merge(right=d.reset_index(), how='left', on='Policy ID')"""

In [ ]:
del policy_data , payment_history , occupation , a , b 
gc.collect()

In [ ]:
test.drop_duplicates(subset ="Policy ID", keep ='first', inplace = True) 

In [ ]:
test = test.fillna(test.median())

In [ ]:
test['PREMIUMDUEDATE_time'].fillna(test['PREMIUMDUEDATE_time'].value_counts().idxmax(), inplace=True)
test['PREMIUMDUEDATE_date'].fillna(test['PREMIUMDUEDATE_date'].value_counts().idxmax(), inplace=True)
test['POSTDATE_time'].fillna(test['POSTDATE_time'].value_counts().idxmax(), inplace=True)
test['POSTDATE_date'].fillna(test['POSTDATE_date'].value_counts().idxmax(), inplace=True)
test['DATEPAID_time'].fillna(test['DATEPAID_time'].value_counts().idxmax(), inplace=True)
test['DATEPAID_date'].fillna(test['DATEPAID_date'].value_counts().idxmax(), inplace=True)
test['PREMIUMDUEDATE'].fillna(test['PREMIUMDUEDATE'].value_counts().idxmax(), inplace=True)
test['POSTDATE'].fillna(test['POSTDATE'].value_counts().idxmax(), inplace=True)
test['DATEPAID'].fillna(test['DATEPAID'].value_counts().idxmax(), inplace=True)

In [ ]:
for col in tn(test.columns):
    if test[col].dtype != 'object':
        test[col]=test[col].astype(object)

In [ ]:
#test = test.astype(object)
test=reduce_mem_usage(test)

In [ ]:
train = train.fillna(train.median())

In [ ]:
train.drop(['index_x'],axis=1,inplace=True)
train.drop(['index_y'],axis=1,inplace=True)
test.drop(['index_x'],axis=1,inplace=True)
test.drop(['index_y'],axis=1,inplace=True)

In [ ]:
train['PREMIUMDUEDATE_time'].fillna(train['PREMIUMDUEDATE_time'].value_counts().idxmax(), inplace=True)
train['PREMIUMDUEDATE_date'].fillna(train['PREMIUMDUEDATE_date'].value_counts().idxmax(), inplace=True)
train['POSTDATE_time'].fillna(train['POSTDATE_time'].value_counts().idxmax(), inplace=True)
train['POSTDATE_date'].fillna(train['POSTDATE_date'].value_counts().idxmax(), inplace=True)
train['DATEPAID_time'].fillna(train['DATEPAID_time'].value_counts().idxmax(), inplace=True)
train['DATEPAID_date'].fillna(train['DATEPAID_date'].value_counts().idxmax(), inplace=True)
train['PREMIUMDUEDATE'].fillna(train['PREMIUMDUEDATE'].value_counts().idxmax(), inplace=True)
train['POSTDATE'].fillna(train['POSTDATE'].value_counts().idxmax(), inplace=True)
train['DATEPAID'].fillna(train['DATEPAID'].value_counts().idxmax(), inplace=True)

In [ ]:
train.drop(['index'],axis=1,inplace=True)
test.drop(['index'],axis=1,inplace=True)

In [ ]:
#test=test.astype(object)
test=reduce_mem_usage(test)
#train=train.astype(object)
train=reduce_mem_usage(train)

In [ ]:
from sklearn import preprocessing
for f in tn(train.columns):
    if train[f].dtype!='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values))  
        train[f] = lbl.transform(list(train[f].values)) 
for f in tn(test.columns):
    if test[f].dtype!='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(test[f].values))  
        test[f] = lbl.transform(list(test[f].values)) 

In [ ]:
Y = train['Lapse']
X = train.drop(['Lapse'],axis=1,inplace=False)
#X=X.astype(float)

In [ ]:
X=X.astype(float)



In [ ]:
params = {
    'objective' :'binary',
    'learning_rate' : 0.01,
    'num_leaves' : 120,
    'feature_fraction': 0.4, 
    'bagging_fraction': 0.4, 
    'bagging_freq':1,
    'boosting_type' : 'gbdt',
    'metric': 'binary_logloss',
    'max_depth' : -1,
    'seed':0,
    'lambda_l2':0.4
}
from sklearn.model_selection import train_test_split
import lightgbm as lgbm

X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y,  random_state=7, test_size=0.33)
    
    # making lgbm datasets for train and valid
d_train = lgbm.Dataset(X_train, Y_train)
d_valid = lgbm.Dataset(X_valid, Y_valid)
    
    # training with early stop
bst = lgbm.train(params, d_train, 500, valid_sets=[d_valid], verbose_eval=50, early_stopping_rounds=100)
    
    # making prediciton for one column
    
preds = bst.predict(test)

In [ ]:
sub['Lapse']=preds
sub.Lapse.value_counts()

In [ ]:
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv" , index=False)
    return FileLink(submission_name+".csv")

In [ ]:
create_submission(sub, 'Zimnat48')

In [ ]:
sub[sub['Lapse']>0.5].shape